In [33]:
import tensorflow as tf

import numpy as np


In [34]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
from IPython.display import clear_output
from six.moves import urllib 

import tensorflow.compat.v2.feature_column as fc 


# Linear Eastimator

In [35]:
#These data sets are records of people who were on the titanic
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') 
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') 
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [36]:
#separating the columns into catagorical and numeric columns
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']
#getting feature columns from the data set
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()  
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
#getting feature names from the data set
for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))


In [37]:
#creating a function that creates the input function
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
    def input_function():  
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  
        if shuffle:
            ds = ds.shuffle(1000) 
            ds = ds.batch(batch_size).repeat(num_epochs)  
        return ds  
    return input_function  
#defining the training and evaluating input functions
train_input_fn = make_input_fn(dftrain, y_train)  
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1)


In [38]:
# defining the linear estimator
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/tmpichltuz3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [39]:
# training the estimator and evaluating the accuracy of it
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print('accuracy:', result['accuracy'])

accuracy: 0.77272725


In [40]:
# you can use this to check the prediction of the survival of a person compared to if they actually survived
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[4])
print('probability of survival:', result[4]['probabilities'][1])
print('Survived (1 = yes, 0 = no): ', y_eval.loc[4])
# 

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/tmpichltuz3/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                          male
age                          34.0
n_siblings_spouses              0
parch                           0
fare                         13.0
class                      Second
deck                            D
embark_town           Southampton
alone                           y
Name: 4, dtype: object
probability of survival: 0.5027193
Survived (1 = yes, 0 = no):  1


# Classification

In [41]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

In [42]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']


In [43]:
#Training and Testing data paths
# The training data is from three types of Iris Flowers, the Iris Setosa, Versicolor, and Virginica
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
#reading the data and assigning it a variable as a pandas dataframe
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# splitting the species name and the data assigned to it into two different dataframes
train_y = train.pop('Species')
test_y = test.pop('Species')

def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


In [44]:
#creating an array of the keys in the training data, so we can input them into the clasifier
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))




In [45]:
#Defining a Deep Neural Network Classifier
classifyer = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns, hidden_units=[30,10], n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/tmpnj5tne1i', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [46]:
#training the classifier, you can adjust the amount of training by adjusting the steps input
# on my device I find the accuracy doesn't increase much past 15000 steps
classifyer.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=15000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/tmpnj5tne1i/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.227018, step = 0
INFO:tensorflow:global_step/sec: 1124.27
INFO:tensorflow:loss = 0.80984163, step = 100 (0.089 sec)
INFO:tensorflow:global_step/sec: 1720.9
INFO:tensorflow:loss = 0.7380511, step = 200 (0.058 sec)
INFO:tensorflow:global_step/sec: 1712.54
INFO:tensorflow:loss = 0.69970775, step = 300 (0.058 sec)
INFO:tensorflow:global_step/sec: 1672.32
INFO:tensorflow:loss = 0.63350385, step = 400 (0.060 sec)
INFO:tensorflow:global_step/sec: 1682.45
INFO:tensorflow:lo

In [47]:
#using tensorflows built in evaluater to see how accurate the classifier is with the testing data set
eval_result = classifyer.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-06-16T15:06:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/tmpnj5tne1i/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.10691s
INFO:tensorflow:Finished evaluation at 2024-06-16-15:06:27
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.96666664, average_loss = 0.11762066, global_step = 15000, loss = 0.11762066
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: /var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/tmpnj5tne1i/model.ckpt-15000

Test set accuracy: 0.967



In [48]:

# this will just take four inputs ('SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth') and output what a prediction for what type of Iris the flower is
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifyer.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))
    

Please type numeric values as prompted.


ValueError: could not convert string to float: ''

In [ ]:
# Use these values to test if the Classifier is accurate
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}